In [ ]:
# I am a beginner please correct me for mistakes 

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(r'/kaggle/input/bank-marketing-dataset/Dataset/bank-additional.csv',delimiter=';')

In [ ]:
#df.head()

In [ ]:
#Dataset has 4119 rows and 21 columns
df.shape

In [ ]:
# no nulls present
df.info()

In [ ]:
df["age"].plot(kind='hist')

In [ ]:
df["job"].value_counts().plot(kind='bar')

In [ ]:
#unabalanced target
print(df["y"].value_counts())
df["y"].value_counts().plot(kind='bar')

In [ ]:
#Using Label encoder 
#selecting all object columns
Object_list = df.select_dtypes(include = "object").columns
print(Object_list)
le = LabelEncoder()
for col in df[Object_list]:
    df[col] = le.fit_transform(df[col].astype(str))

In [ ]:
# Get highly correlated variables to drop. 
#reference code: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/

corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.70
to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]
print("Columns to drop:",to_drop)

In [ ]:
df.drop(to_drop,inplace=True,axis=1)
X=df.drop('y',axis=1)
y=df['y']

In [ ]:
#Split data for train and test
xtrain,xtest,ytrain,ytest= train_test_split(X,y,test_size=0.2)

In [ ]:
clf = LogisticRegression(random_state=0)
clf.fit(xtrain,ytrain)

In [ ]:
ypredict=clf.predict(xtest)
print("Score",clf.score(xtest,ytest))
print("Confusion matrix\n",metrics.confusion_matrix(ytest,ypredict))
print("\nAUCscore:",round(metrics.roc_auc_score(ytest,ypredict)*100,3),"%")

In [ ]:
print(metrics.classification_report(ytest,ypredict))

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
roc_auc= cross_val_score(clf, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print("roc_auc scores in Repeated K fold", round(mean(roc_auc)*100,3))